In [1]:
from KmeansPPcenters import KMeanPlusPlus
from Gonzalez_centers import Gonzalez
from sklearn.cluster import KMeans
from scipy.spatial import distance
import numpy as np
import pandas as pd
import random
import scipy.io as scio
from numpy import genfromtxt
from KMeansOut import kmeansOutliers, cost
from sklearn.metrics import average_precision_score, precision_recall_curve
from Noise import add_random_noise, compute_phi_star, cost, add_random_noise_th

In [2]:
dirNam = "realData/Skin/"
proDir = "realDataProcessed/"

#Processing skin
fileName = "Skin_NonSkin.txt"

skin = np.genfromtxt(dirNam + fileName, delimiter='')

np.savetxt(proDir + "skin.csv", skin, delimiter=",",  fmt='%f')

In [3]:
skin_data = genfromtxt('realDataProcessed/skin.csv', delimiter=',')
skin_data=skin_data[:,0:3] #remove the labels


In [4]:
data= skin_data
num_clusters= 10
random_indx= random.randint(0, len(data)-1)
init=data[random_indx]

#Get KPP centers
KPP=KMeanPlusPlus(num_clusters=num_clusters, init=init)
KPP.fit(data)
KPP_centers= KPP.centers




In [5]:
num_clusters=[10, 20]
zs =[25, 50, 100]
min_values=[0, 100]
max_values= [200,255]
itr=100
tol=.05

In [6]:
for num_cluster in num_clusters:
    for z in zs:
        for min_value in min_values:
            for max_value in max_values:
                #Step 1: Adding noise
                #random.seed(z)
                print("num_cluster:{}, z:{}, min_value:{}, max_value:{}".format(num_cluster, z, min_value, max_value))
                data_with_outliers, z_indx = add_random_noise_th(data, z, min_value, max_value)
                data_inliers= np.delete(data, z_indx, axis=0)
                
                #Step 2:random intialization for KPP
                random_indx= random.randint(0, len(data_with_outliers)-1)
                init=data_with_outliers[random_indx]
                                
                # Step 3: KPP initialization for LLoyd
                KPP=KMeanPlusPlus(num_clusters=num_cluster, init=init)
                KPP.fit(data_with_outliers)
                centers= KPP.centers
                #print(type(centers))
                #print(centers.shape)
                new_centers=np.zeros((num_cluster, 3))
                #print("new_centers:{}".format(new_centers))
                #convergence of LLoyd
                
                for i in range(itr):
                    #print(i)
                    
                    #centers= skin_data[np.random.choice(np.arange(len(data_with_outliers)-1), size= 10)]
                    #Step 4: Compute distance between all centers and points 
                    dist= distance.cdist(data_with_outliers, centers)
                    dist = np.amin(dist, axis = 1)
                    
                    #Step 5: Remove furthest z points
                    indx_list = np.argpartition(dist, -z)[-z:]
                    data_new = np.delete(data, indx_list, axis=0)
                    
                    #Step 6: Compute distance with data_new and centers & save the index of min distance in cid
                    dist_new = distance.cdist(data_new, centers)
                    cid = np.argmin(dist_new, axis=1)
                    dist_new = np.amin(dist_new, axis = 1)
                    
                    #Step 7: Compute new centers as mean of data points in each clusters
                    for j in range(num_cluster):
                        if len(data_new[cid==j])>0:
                            new_centers[j]= np.mean(data_new[cid==j], axis=0)
                            #print(new_centers[j])
                    #print("New centers now:{}".format(new_centers))
                    #new_centers= skin_data[np.random.choice(np.arange(len(data_with_outliers)-1), size= 10)]
                    
                    
                    
                    #Generate set them as new centers
                    old_centers= centers
                    centers= new_centers
                    
                    #Step 8: Check the difference in the set of centers
                    isOPTIMAL= True

                    if (len(np.setdiff1d(new_centers,old_centers)))>tol:
                        isOPTIMAL= False
                    
                    #Step 9: Stop if value less than tolerence
                    if isOPTIMAL:
                        break
                    
                    
                #Step 10: Compute precision and recall
                #z_indx : Actual outliers
                precision = len(z_indx)/(len(z_indx)+ len(np.setdiff1d(z_indx, indx_list)))
                recall= precision = len(z_indx)/(len(z_indx)+ len(np.setdiff1d(indx_list, z_indx)))
                
                #Step 11: Cost
                #LO_cost, index= cost(data_with_outliers, cid, centers, len(indx_list))
                #print(LO_cost)
                #x1= KPP.predict(data_with_outliers)
                print(("Precision:{}, recall:{}". format(precision, recall)))
                

num_cluster:10, z:25, min_value:0, max_value:200
Precision:0.5, recall:0.5
num_cluster:10, z:25, min_value:0, max_value:255
Precision:0.5, recall:0.5
num_cluster:10, z:25, min_value:100, max_value:200
Precision:0.5, recall:0.5
num_cluster:10, z:25, min_value:100, max_value:255
Precision:0.5, recall:0.5
num_cluster:10, z:50, min_value:0, max_value:200
Precision:0.5, recall:0.5
num_cluster:10, z:50, min_value:0, max_value:255
Precision:0.5, recall:0.5
num_cluster:10, z:50, min_value:100, max_value:200
Precision:0.5, recall:0.5
num_cluster:10, z:50, min_value:100, max_value:255
Precision:0.5, recall:0.5
num_cluster:10, z:100, min_value:0, max_value:200
Precision:0.5, recall:0.5
num_cluster:10, z:100, min_value:0, max_value:255
Precision:0.5, recall:0.5
num_cluster:10, z:100, min_value:100, max_value:200
Precision:0.5, recall:0.5
num_cluster:10, z:100, min_value:100, max_value:255
Precision:0.5, recall:0.5
num_cluster:20, z:25, min_value:0, max_value:200
Precision:0.5, recall:0.5
num_clust